In [8]:
import os
import glob
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import tqdm

In [9]:
df = pd.read_csv('../data/prof_trainigns_df.csv',index_col=0,parse_dates=True)
df = df.loc[df.index.year > 1968]
df = df.interpolate(method = 'time').drop(columns = "time_as_number")
df

,SECCI,Temperatur,Salinität,SiO4,PO4,NO2,NO3,NOx,NH4
Datum,,,,,,,,,
1969-01-02,NaN,5.30,32.5520,NaN,NaN,NaN,NaN,NaN,NaN
1969-01-03,2.0,5.20,32.4270,8.40,0.76,1.120,3.57,NaN,13.60
1969-01-06,2.5,4.80,32.6520,11.60,0.84,1.110,6.07,NaN,5.90
1969-01-07,2.5,4.55,32.7025,11.35,0.87,1.195,6.14,NaN,7.00
1969-01-08,2.5,4.30,32.7530,11.10,0.90,1.280,6.21,NaN,8.10
...,...,...,...,...,...,...,...,...,...
2010-12-21,2.6,4.00,30.8610,11.11,0.63,0.520,18.85,19.37,2.39
2010-12-22,2.7,4.20,31.6430,11.03,0.55,0.440,12.98,13.43,2.45
2010-12-28,1.9,2.90,30.1370,18.46,0.93,0.810,26.10,26.91,3.64


In [10]:


means = pd.read_json("../data/bbdc_prof/means.json",typ='series')
variance = pd.read_json("../data/bbdc_prof/variances.json",typ='series')

columns = means.index.tolist()
df[columns] = (df[columns]- np.array(means)) / np.sqrt(np.array(variance))


In [11]:
from sklearn.ensemble import GradientBoostingRegressor
val_df = df.loc[(df.index.year == 2005) | (df.index.year == 2006)][["Temperatur"	,"Salinität", "NO2","NO3","NOx"]]
train_df = df.loc[df.index.year > 2006][["Temperatur"	,"Salinität", "NO2","NO3","NOx"]]

In [12]:
val_x = val_df.drop(columns="NOx")
val_y = val_df.NOx
train_x =  train_df.drop(columns="NOx")
train_y = train_df.NOx


switcher = GradientBoostingRegressor(n_estimators=100,random_state=0
)

switcher.fit(train_x,train_y)

vals = switcher.predict(val_x)
np.sqrt(np.mean(np.square(vals - val_y)))

0.057165963008632546

In [13]:
train = df.loc[df.index.year < 2004]

df.loc[df.index.year < 2004,'NOx'] = switcher.predict(train[["Temperatur"	,"Salinität", "NO2","NO3"]].fillna(method='bfill'))